# Inteligencja Obliczeniowa - Projekt
## Temat: Zgłębianie danych na przykładzie wybranej bazy danych
### Jakub Sikorski


# Wstęp

W tym projekcie zastosuję kilka metod (m.in drzewa decyzyjne, klasyfikacje k najbliższych sąsiadów i naiwną bayesowską) do zgłębiania bazy danych. Za bazę danych posłuży mi baza "Breast Cancer Wisconsin" (4b),
czyli baza dotycząca raka piersi (już zdiagnozowanego) w Wisconsin.

Projekt ten służy głównie do nauczenia się korzystania z bibliotek dostępnych w Pythonie. Biblioteki, które były mi potrzebne przy tym projekcie to:
1. pandas - do wczytania i manipulacji danych,
2. scikit-learn - potrzebna do przeprowadzenia klasyfikacji i zmierzenia ich dokładności, a także do podziału danych na dane treningowe i testowe.


In [1]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn import metrics

from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
import graphviz

from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

# Dane

Baza danych "Breast Cancer Wisconsin" jest bazą danych publicznie [dostępną](https://archive.ics.uci.edu/ml/datasets/breast+cancer+wisconsin+(original) ). Składa się ona z 699 rekordów i 11 atrybutów. W bazie danych znajdują się też brakujące dane.

Kolumny opisują następująco:
1. Numer ID próbki
2. Clump Thickness: 1 - 10, czyli grubość "grudki"
3. Uniformity of Cell Size: 1 - 10, czyli jednorodność wielkości komórek w próbce
4. Uniformity of Cell Shape: 1 - 10, czyli jednorodność kształtu komórek w próbce
5. Marginal Adhesion: 1 - 10, mówi o tym ile komórek poza tkanką nabłonkową się ze sobą skleja
6. Single Epithelial Cell Size: 1 - 10, wielkość pojedynczych komórek nabłonka
7. Bare Nuclei: 1 - 10, proporcja komórek nie otoczonych cytoplazma w stosunku do tych otoczonych
8. Bland Chromatin: 1 - 10, mówi o strukturze (budowie) jądra
9. Normal Nucleoli: 1 - 10, mówi o wielkości jąderka
10. Mitoses: 1 - 10, mówi o aktywności mitotycznej
11. Class: (2 for benign, 4 for malignant) klasyfikuje czy jest to nowotwór łagodny czy złośliwy

In [2]:
missing_values = ["n/a", "na", "--", "?", "NaN"]
df = pd.read_csv('breast-cancer-wisconsin.data', na_values=missing_values)
df_changed = df

### Przykładowe dane wyglądają następująco:

In [3]:
df.head(6)

,id,clump-thickness,uniformity-of-cell-size,uniformity-of-cell-shape,marginal-adhesion,single-epithelial-cell-size,bare-nuclei,bland-chromatin,normal-nucleoli,mitoses,class
0,1000025,5,1,1,1,2,1.0,3,1,1,2
1,1002945,5,4,4,5,7,10.0,3,2,1,2
2,1015425,3,1,1,1,2,2.0,3,1,1,2
3,1016277,6,8,8,1,3,4.0,3,7,1,2
4,1017023,4,1,1,3,2,1.0,3,1,1,2
5,1017122,8,10,10,8,7,10.0,9,7,1,4


Pierwszą rzeczą, którą postanowiłem zmienic to zamienić kolumnę "class" na 0 (łagodny) i 1 (złośliwy)

In [4]:
for index, row in enumerate(df_changed['class']):
    if row == 2:
        df_changed.at[index, 'class'] = 0
    if row == 4:
        df_changed.at[index, 'class'] = 1

Po zamianie "klasy" nowotworu następnym krokiem była zmiana lub usunięcie brakujących danych. W oryginalnych danych brak wartości był oznaczany przez "?", po przepisaniu do nowej zmiennej było to NaN.

Łącznie takich pól, które nie miały wartości było 16 i wszystkie znajdowały się w kolumnie "bare-nuclei". Wszystkie kolumny (oprócz id i typu) mogą przyjmować wartości od 1 do 10 i należy je rozumieć jako skalę, np. dla kolumny _mitoses_: 1 - prawie brak aktywności mitotycznej, 10 - bardzo duża aktywność mitotyczna.

W związku z tym nie mogłem uzupełnić danych ani przez medianę ani przez średnią, więc rekordy z brakującymi danymi usunąłem z danych.

In [5]:
df_changed.iloc[23]

id                             1057013.0
clump-thickness                      8.0
uniformity-of-cell-size              4.0
uniformity-of-cell-shape             5.0
marginal-adhesion                    1.0
single-epithelial-cell-size          2.0
bare-nuclei                          NaN
bland-chromatin                      7.0
normal-nucleoli                      3.0
mitoses                              1.0
class                                1.0
Name: 23, dtype: float64

In [6]:
print (df_changed.isnull().sum())

id                              0
clump-thickness                 0
uniformity-of-cell-size         0
uniformity-of-cell-shape        0
marginal-adhesion               0
single-epithelial-cell-size     0
bare-nuclei                    16
bland-chromatin                 0
normal-nucleoli                 0
mitoses                         0
class                           0
dtype: int64


In [7]:
df_changed = df_changed.dropna()

In [8]:
print(df_changed.isnull().sum())

id                             0
clump-thickness                0
uniformity-of-cell-size        0
uniformity-of-cell-shape       0
marginal-adhesion              0
single-epithelial-cell-size    0
bare-nuclei                    0
bland-chromatin                0
normal-nucleoli                0
mitoses                        0
class                          0
dtype: int64


In [9]:
df_changed.to_csv('breast-cancer-wyczyszczone.csv', index=False)

In [10]:
breast_cancer = pd.read_csv('breast-cancer-wyczyszczone.csv')

In [11]:
breast_cancer.describe()

,id,clump-thickness,uniformity-of-cell-size,uniformity-of-cell-shape,marginal-adhesion,single-epithelial-cell-size,bare-nuclei,bland-chromatin,normal-nucleoli,mitoses,class
count,6.830000e+02,683.000000,683.000000,683.000000,683.000000,683.000000,683.000000,683.000000,683.000000,683.000000,683.000000
mean,1.076720e+06,4.442167,3.150805,3.215227,2.830161,3.234261,3.544656,3.445095,2.869693,1.603221,0.349927
std,6.206440e+05,2.820761,3.065145,2.988581,2.864562,2.223085,3.643857,2.449697,3.052666,1.732674,0.477296
min,6.337500e+04,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
25%,8.776170e+05,2.000000,1.000000,1.000000,1.000000,2.000000,1.000000,2.000000,1.000000,1.000000,0.000000
50%,1.171795e+06,4.000000,1.000000,1.000000,1.000000,2.000000,1.000000,3.000000,1.000000,1.000000,0.000000
75%,1.238705e+06,6.000000,5.000000,5.000000,4.000000,4.000000,6.000000,5.000000,4.000000,1.000000,1.000000
max,1.345435e+07,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,1.000000


### Po wyczyszczeniu danych podzieliłem je losowo na dane treningowe (70%) i dane testowe (30%).

Następnie przystąpiłem do klasyfikacji przy użyciu drzew decyzyjnych.

In [12]:
X = breast_cancer.values[:, 1:10]
Y = breast_cancer.values[:,-1]

(X_train, X_test, Y_train, Y_test) = train_test_split(X, Y, train_size=0.7, test_size = 0.3, random_state=1)

## Drzewa decyzyjne

Drzewa decyzyjne są graficzną metodą wspomagania procesu decyzyjnego. Jest to jedna z
najczęściej wykorzystywanych technik analizy danych. Drzewo składają się z korzenia oraz
gałęzi prowadzących z korzenia do kolejnych wierzchołków. Wierzchołki, z których wychodzi
co najmniej jedna krawędź, są nazywane węzłami, a pozostałe wierzchołki – liśćmi. W
każdym węźle sprawdzany jest pewien warunek dotyczący danej obserwacji, i na jego
podstawie wybierana jest jedną z gałęzi prowadząca do kolejnego wierzchołka. Klasyfikacja
danej obserwacji polega na przejściu od korzenia do liścia i przypisaniu do tej obserwacji
klasy zapisanej w danym liściu. [źródło](http://home.agh.edu.pl/~pmarynow/pliki/iwmet/drzewa.pdf)


Dokładnośc drzewa decyzyjnego przedstawiona na macierzy błędu. (wyjaśnienie i porównanie poźniej)

In [13]:
dtc = DecisionTreeClassifier()
y_test = dtc.fit(X_train, Y_train)
y_pred = dtc.predict(X_test)

print('\n',metrics.confusion_matrix(Y_test, y_pred))

dot_data = export_graphviz(dtc, out_file=None) 
graph = graphviz.Source(dot_data)
#graph


 [[129   4]
 [  4  68]]


## Klasyfikator k najbliższych sąsiadów

Dla klasyfikatora tego ustalamy k (czyli ilość "sąsiadów"), np. k = 2. Następnie dla każdego rekordu, który chcemy sklasyfikować wyszukujemy 3 najbardziej podobne(w tym projekcie jest to przez metrykę euklidesową) rekordy. Rekord który chcemy sklasyfikować przyjmuje taką klasę jak większość z wybranych, podobnych, rekordów. [źródło - zajęcia]

Porównam tu ze sobą klasyfikatory kNN, k = 3 i kNN, k = 10, aby sprawdzić czy większa ilość sąsiadów wpłynie na dokładność.

In [14]:
knn3 = KNeighborsClassifier(n_neighbors=3, metric='euclidean')
knn3.fit(X_train, Y_train)
y_pred3 = knn3.predict(X_test)
print("\nkNN-3\n",metrics.confusion_matrix(Y_test, y_pred3))

knn10 = KNeighborsClassifier(n_neighbors=10, metric='euclidean')
knn10.fit(X_train, Y_train)
y_pred10 = knn10.predict(X_test)
print("\nkNN-10\n", metrics.confusion_matrix(Y_test, y_pred10))


kNN-3
 [[128   5]
 [  2  70]]

kNN-10
 [[129   4]
 [  3  69]]


## Naiwny klasyfikator bayesowski

Klasyfikator naiwny bayesowski (Naive Bayes) to klasyfikator probabilistyczny.
Bierzemy wartość pierwszej danej z badanego rekordu, następnie sprawdzamy czy
inne rekordy z tą samą wartością „siedziały” w jednej klasie, czy drugiej. Robimy tak dla
wszystkich kolejnych wartości. Następnie wykorzystując wzór Bayesa na
prawdopodobieństwo warunkowe oraz zakładając, że dane z kolumn są niezależne
(stąd naiwność) szacujemy, w której klasie najprawdopodobniej znajduje się nasz rekord. [źródło - zajęcia]

In [15]:
model = GaussianNB()
model.fit(X_train, Y_train)
pred_bayes = model.predict(X_test)
#kolumny to wartosci przewidziane, wiersze to wartosci faktyczne
print("\nBayes\n",metrics.confusion_matrix(Y_test, pred_bayes))


Bayes
 [[127   6]
 [  0  72]]


In [16]:
score_knn3 = metrics.accuracy_score(Y_test, y_pred3)
score_knn10 = metrics.accuracy_score(Y_test, y_pred10)
score_bayes = metrics.accuracy_score(Y_test, pred_bayes)
score_tree = metrics.accuracy_score(Y_test, y_pred)

scores = [score_knn3*100, score_knn10*100, score_bayes*100, score_tree*100]

print("\nDokładność kNN-3:", score_knn3)
print("Dokładność kNN-10:", score_knn10)
print("Dokładność Bayes:", score_bayes)
print("Dokładność drzewa decyzyjne:", score_tree)


Dokładność kNN-3: 0.9658536585365853
Dokładność kNN-10: 0.9658536585365853
Dokładność Bayes: 0.9707317073170731
Dokładność drzewa decyzyjne: 0.9609756097560975


# Wyniki

Dla wszystkich klasyfikatorów dokładność wyszła bardzo wysoka, powyżej 95%. Oznacza to tyle, że z dostarczonych w tej bazie danych można z dość sporą pewnością określić czy nowotwór jest łagodny czy złośliwy.

In [17]:
print("\nBayes\n",pd.crosstab(Y_test, pred_bayes, rownames=["test"] ,colnames=["pred"]))


Bayes
 pred  0.0  1.0
test          
0.0   127    6
1.0     0   72


Jeden z klasyfikatorów się jednak wyróżnia. W przypadku naiwnego klasyfikatora bayesowkiego, analizując macierz błędu nie ma ani jednego przypadku, w którym przewidziana wartość to guz łagodny (0), gdy w rzeczywistości jest on złośliwy (1) (False Negative).

Porównując klasyfikatory k najbliższych sąsiadów, w przypadku wyznaczenia k=10 popełnił on 1 (FP) błąd mniej niż przy k=3. Oba jednak mają bardzo wysoką dokładność.